<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Intro  
  
In this notebook, we will convert addresses into latitude and longitude values.  We will use the Foursquare API to explore a few neighborhoods in Toronto.  We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters.  We will use the k-means algorithm to complete this task.  Finally, we will use the Folium library to visualize the neighborhoods in Toronto and their clusters.  

First lets import the necessary libraries

In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import os
from dotenv import load_dotenv

### Load our data 

In [2]:
df = pd.read_csv('./Data/geo_df.csv')

In [4]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Use geopy library to get latitude and longitude values for Toronto

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geographical coordinate of Toronto is {latitude}, {longitude}')

The geographical coordinate of Toronto is 43.6534817, -79.3839347


### Create a map of Toronto with neighborhoods superimposed on top

In [6]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = f'Neighborhoods:{neighborhood}, Borough:{borough}'
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

In [7]:
map_toronto

### Lets make our map a little less crowded and only choose the boroughs that have the word Toronto in them

In [10]:
df_2 = df[df["Borough"].str.contains('Toronto|toronto')].reset_index(drop = True)
df_2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Visualize these neighborhoods

In [11]:
map_toronto_boroughs = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_boroughs)  


In [13]:
map_toronto_boroughs

### Define Foursquare credentials

In [15]:
load_dotenv()
client_id = os.getenv(client_id)
client_secret = os.getenv(client_secret)

NameError: name 'client_id' is not defined